In [5]:
import cv2
import numpy as np
import os

In [6]:
dataset_path = '{}\\dataset_out'.format(os.getcwd().replace('\\auxiliaries',''))

In [7]:
list_files = []

for file in os.listdir(dataset_path):
    list_files.append(int(file.replace('.png','')))

list_files = np.sort(list_frames)

In [16]:
img_array = []
size = None

for file in list_files:
    filename = '{}\\{}.png'.format(dataset_path, file)
    img = cv2.imread(filename)
    img_array.append(img)
    
    if size is None:
        height, width, layers = img.shape
        size = (width,height)

In [17]:
out = cv2.VideoWriter('{}\\project.avi'.format(dataset_path),cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()